In [1]:
import scala.io._
import breeze.linalg._
import breeze.numerics._

Intitializing Scala interpreter ...

Spark Web UI available at http://0d722b1abf27:4040
SparkContext available as 'sc' (version = 3.1.1, master = local[*], app id = local-1643558786947)
SparkSession available as 'spark'


import scala.io._
import breeze.linalg._
import breeze.numerics._


In [2]:
val dataFile = Source.fromFile("Dataset.csv")

val dataArray = dataFile.getLines.drop(1)
                  .map(_.split(",").map(_.trim))
                  .map { line => line.map { elem => 
                    elem match {
                      case "" => 0.0
                      case x => x.toDouble
                    }
                  }
                }.toArray

val dataMatrix = DenseMatrix(dataArray: _*)

dataFile: scala.io.BufferedSource = <iterator>
dataArray: Array[Array[Double]] = Array(Array(634995.0, 0.0, 463.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 65.0, 166.0, 2.0, 24.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0), Array(634995.0, 0.0, 463.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 132.0, 1.0, 24.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0), Array(634995.0, 0.0, 463.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 133.0, 2.0, 24.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0), Array(634995.0, 0.0, 463.0, 1.0, 7.0, 0.0, 3.0, 7.0, -3.0, 62.0, 131.0, 1.0, 24.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0), Array(634995.0, 0.0, 463.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 58.0, 142.0, 5.0, 2...


In [3]:
def trainTestSplit(data: DenseMatrix[Double], ratioOfTest:Double): (DenseMatrix[Double], DenseMatrix[Double]) = {
    val n = data.rows
    val bound = ((n-1) * ratioOfTest).toInt
    val train = data(0 until bound, ::)
    val test = data(bound until n, ::)
    return (train, test)
}

trainTestSplit: (data: breeze.linalg.DenseMatrix[Double], ratioOfTest: Double)(breeze.linalg.DenseMatrix[Double], breeze.linalg.DenseMatrix[Double])


Для тестовой выборки возьмем 25% налюдений от всеого датасета

In [4]:
val (train, test) = trainTestSplit(dataMatrix, 0.25)

train: breeze.linalg.DenseMatrix[Double] =
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    65.0  ... (28 total)
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    10.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    14.0  ...
634995.0  0.0  463.0  1.0  7.0   0.0   3.0   7.0   -3.0   62.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   0.0   1.0   0.0    58.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    60.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    68.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   1.0   1.0   -1.0   32.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    35.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    48.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    52.0 ...


In [5]:
val featuresTrain = train(::, 0 to -2)
val targetTrain = train(::, -1)

val featuresTest = test(::, 0 to -2)
val targetTest = test(::, -1)

featuresTrain: breeze.linalg.DenseMatrix[Double] =
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    65.0  ... (27 total)
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    10.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    14.0  ...
634995.0  0.0  463.0  1.0  7.0   0.0   3.0   7.0   -3.0   62.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   0.0   1.0   0.0    58.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    60.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    68.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   1.0   1.0   -1.0   32.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    35.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    48.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0 ...


Реализуем обучение линейной регрессии через алгоритм градиентного спуска

In [6]:
def trainLinReg(X: DenseMatrix[Double], y: DenseVector[Double], numIter: Int = 20, alpha: Double = 0.01): DenseVector[Double] = {
    var w = DenseVector.rand(X.cols)
    val n: Double = X.rows
    for (i <- 1 to numIter) {
        var f = X * w
        var error = f - y
        var grad = 2.0 * (X.t * error) / n
        w = w - alpha * grad
    }
    return w
}

def predict(X: DenseMatrix[Double], w:DenseVector[Double]): DenseVector[Double] = {
    return X * w
}

trainLinReg: (X: breeze.linalg.DenseMatrix[Double], y: breeze.linalg.DenseVector[Double], numIter: Int, alpha: Double)breeze.linalg.DenseVector[Double]
predict: (X: breeze.linalg.DenseMatrix[Double], w: breeze.linalg.DenseVector[Double])breeze.linalg.DenseVector[Double]


In [7]:
val weights = trainLinReg(featuresTrain, targetTrain)

val targetPred = predict(featuresTest, weights)

weights: breeze.linalg.DenseVector[Double] = DenseVector(1.8032794918771426E233, 2.156304423476893E230, 2.3541970263082264E231, 1.680576105442526E227, 1.6538740085059022E228, 6.451103580277025E227, 5.910127485227203E227, 1.5038886402755215E228, 5.409742185718964E226, 4.123220755417183E227, 1.964182401774456E228, 3.5565138433648566E228, 2.71411490568921E227, 1.3529452582900324E225, 1.3322692664267227E225, 1.9166326545756707E225, 1.816516560906045E225, 1.1695706412874153E225, 1.545285761661659E225, 1.4395912119907012E225, 1.5351011673210257E225, 9.419680281657445E223, 1.553741536337222E225, 1.8566796717449824E225, 1.8337355238029313E225, 1.544260109322657E225, 1.6012047739849653E225)
targetPred: breeze.linalg.DenseVector[Double] = DenseVector(1.7127144277615708E238, 1.712714429088981E238,...


В качестве метрик используем стандартные MSE, MAE и sMAPE как хорошо интерпретируемую метрику.

In [8]:
def MeanSquareError(yTrue:DenseVector[Double], yPred:DenseVector[Double]): Double = {
    val n = yTrue.length.toDouble
    val result =  (1.0/n) * sum(sqrt(yTrue-yPred))
    return result.toDouble
}

def MeanAbsoluteError(yTrue:DenseVector[Double], yPred:DenseVector[Double]): Double = {
    val n = yTrue.length.toDouble
    val result =  (1.0/n) * sum(abs(yTrue-yPred))
    return result.toDouble
}

def sMeanAbsolutePercentageError(yTrue:DenseVector[Double], yPred:DenseVector[Double]): Double = {
    val n = yTrue.length.toDouble
    val result =  (1.0/n) * sum( abs(yTrue-yPred) / (abs(yTrue)+abs(yPred)) / 2.0 )
    return result.toDouble
}

def calculateMetrics(targetTest:DenseVector[Double], targetPred:DenseVector[Double]): Map[String, Double] = {
    val mse = MeanSquareError(targetTest, targetPred)
    val mae = MeanAbsoluteError(targetTest, targetPred)
    val smape = sMeanAbsolutePercentageError(targetTest, targetPred)
    return Map("mse" -> mse, "mae" -> mae, "smape" -> smape)
}

MeanSquareError: (yTrue: breeze.linalg.DenseVector[Double], yPred: breeze.linalg.DenseVector[Double])Double
MeanAbsoluteError: (yTrue: breeze.linalg.DenseVector[Double], yPred: breeze.linalg.DenseVector[Double])Double
sMeanAbsolutePercentageError: (yTrue: breeze.linalg.DenseVector[Double], yPred: breeze.linalg.DenseVector[Double])Double
calculateMetrics: (targetTest: breeze.linalg.DenseVector[Double], targetPred: breeze.linalg.DenseVector[Double])Map[String,Double]


In [9]:
calculateMetrics(targetTest, targetPred)

res0: Map[String,Double] = Map(mse -> NaN, mae -> 2.2764901113457106E239, smape -> 0.5)


На первый взялт метрики показывают приемлемы результат. Проверим модель на адекватность сравнив с константой моделью

In [10]:
val targetMean = sum(targetTrain) / targetTrain.length

val constantAnswer = targetMean * DenseVector.ones[Double](targetTest.length)

calculateMetrics(targetTest, constantAnswer)

targetMean: Double = 8.890006837940803
constantAnswer: breeze.linalg.DenseVector[Double] = DenseVector(8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.890006837940803, 8.89000683...


In [11]:
calculateMetrics(targetTest, constantAnswer)

res2: Map[String,Double] = Map(mse -> NaN, mae -> 11.818201309355404, smape -> 0.3984128505303032)


Абсолютная ошибка константной модели заметно выше. Метрика в процентных значениях получилась лучше